# Project for Data Science Certifications

import libraries

In [8]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

read table with postal codes from wiki and convert to dataframe

In [3]:
link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

tables = pd.read_html(link)

df = tables[0].iloc[1:]
df.columns = ['Postal Code','Borough','Neighbourhood']
df.head()

,Postal Code,Borough,Neighbourhood
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"


delete rows with not assigned neighbourhood

In [4]:
df = df[df['Neighbourhood'] != 'Not assigned']
df['ni']= range(0, len(df)) # this is the line that generates the warning
df = df.set_index('ni')

read csv file with geospatial coordinates and added coordinates from this table to df table:

In [5]:
df1 = pd.read_csv("Geospatial_Coordinates.csv")
df1.head()

# join my tables df and df1 to df (added coordinates to df table):

df['Latitude'] = None
df['Longitude'] = None

for i in range(0, len(df)):
    for j in range(0, len(df1)):
        if df['Postal Code'][i] == df1['Postal Code'][j]:
            df['Latitude'][i] = df1['Latitude'][j]
            df['Longitude'][i] = df1['Longitude'][j]       

only Toronto:

In [6]:
df = df[df['Borough'].isin(['Downtown Toronto','Central Toronto','West Toronto','East Toronto'])]

df['ni']= range(0, len(df)) # this is the line that generates the warning
df = df.set_index('ni')
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
ni,,,,,
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623,-79.3895
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3789
3,M5C,Downtown Toronto,St. James Town,43.6515,-79.3754
4,M4E,East Toronto,The Beaches,43.6764,-79.293


Use geopy library to get the latitude and longitude values of Toronto.
In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent ny_explorer, as shown below.

In [9]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 43.6534817, -79.3839347.


Create a map of New York with neighborhoods superimposed on top.

In [ ]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Define Foursquare Credentials and Version

In [10]:
CLIENT_ID = '1QP0KV505HFG3NA0TFOZVCBIPVL3HUD0FWJGICIET5MY5FV0' # your Foursquare ID
CLIENT_SECRET = 'NFXYW2PRH3V5LDQXL1UXCF335ZEMQHICXNGL00HRUDQQLHSL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1QP0KV505HFG3NA0TFOZVCBIPVL3HUD0FWJGICIET5MY5FV0
CLIENT_SECRET:NFXYW2PRH3V5LDQXL1UXCF335ZEMQHICXNGL00HRUDQQLHSL


In [11]:
df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
ni,,,,,
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623,-79.3895
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3789
3,M5C,Downtown Toronto,St. James Town,43.6515,-79.3754
4,M4E,East Toronto,The Beaches,43.6764,-79.293
5,M5E,Downtown Toronto,Berczy Park,43.6448,-79.3733
6,M5G,Downtown Toronto,Central Bay Street,43.658,-79.3874
7,M6G,Downtown Toronto,Christie,43.6695,-79.4226
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.6506,-79.3846


Downtown_Toronto:

In [12]:
downtown_data = df[df['Borough'] == 'Downtown Toronto']

Let's explore the first neighborhood in our dataframe.
Get the neighborhood's name.

In [15]:
downtown_data.loc[0, 'Neighbourhood']

'Regent Park, Harbourfront'

Get the neighborhood's latitude and longitude values.

In [16]:
neighborhood_latitude = downtown_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = downtown_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = downtown_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


Now, let's get the top 100 venues that are in 'Regent Park, Harbourfront' within a radius of 500 meters.

In [17]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=1QP0KV505HFG3NA0TFOZVCBIPVL3HUD0FWJGICIET5MY5FV0&client_secret=NFXYW2PRH3V5LDQXL1UXCF335ZEMQHICXNGL00HRUDQQLHSL&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100'